### The following notebook is an implementation of content based recommendation system, 
Content based recommendations are generated based on the simalirity between Item pairs, First document vectors and lebel encodings are generated for the item features, then these vectors are used to calculate cosine similarity, pearson correlation or euclidian distance (any of these methods)

On generating a similarity matrix we can easily sort most similar items by ID

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_dir = "Mldata/"

In [31]:
users = pd.read_csv(data_dir+"users.csv")
posts = pd.read_csv(data_dir+"posts.csv")
views = pd.read_csv(data_dir+"views.csv")

In [4]:
users.head()

,_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [5]:
posts.head()

,_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [6]:
views.head()

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


In [7]:
#CONTENT BASED 
#SMILAR BLOG POSTS

In [8]:
posts.category.value_counts()

Photography                                                                                                             81
Drawings                                                                                                                47
Painting                                                                                                                20
Visual Arts                                                                                                             10
Computer Technology|Machine Learning                                                                                     7
                                                                                                                        ..
Computer Technology|Web designing|Artificial Intelligence|Computer Application|Machine Learning|Frontend Development     1
Craft|Drawings|Conceptual|2D Composition                                                                                 1
Drawings|Waterco

In [9]:
posts[' post_type'].value_counts()

artwork    241
blog       198
project     27
skill       27
Name:  post_type, dtype: int64

In [10]:
ser = posts.category

In [11]:
ser = ser.str.split('|', expand=False)
ser.fillna(value=np.nan, inplace=True)
ser

0                                  [Plant Biotechnology]
1      [Artificial Intelligence, Machine Learning, In...
2                                    [Operating Systems]
3                                             [Drawings]
4                                     [Competition Laws]
                             ...                        
488        [Computer Technology, Information Technology]
489    [Computer Technology, Computer Application, In...
490                                                  NaN
491                                [Science; Technology]
492    [Computer Technology, Computer Application, In...
Name: category, Length: 493, dtype: object

In [12]:
# ser.columns

In [13]:
posts = pd.concat([posts, ser.rename('cat_multilabel')], axis=1)
posts.fillna(value=np.nan, inplace=True)

In [14]:
posts._id.nunique()

493

In [15]:
posts['cat_multilabel'] = posts['cat_multilabel'].apply(lambda d: d if isinstance(d, list) else [])

In [16]:
posts['cat_multilabel']

0                                  [Plant Biotechnology]
1      [Artificial Intelligence, Machine Learning, In...
2                                    [Operating Systems]
3                                             [Drawings]
4                                     [Competition Laws]
                             ...                        
488        [Computer Technology, Information Technology]
489    [Computer Technology, Computer Application, In...
490                                                   []
491                                [Science; Technology]
492    [Computer Technology, Computer Application, In...
Name: cat_multilabel, Length: 493, dtype: object

#### Creating one hot encodings

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()
one_hot.fit([posts[' post_type']])
post_type_enc = one_hot.fit_transform([posts[' post_type']])

one_hot_multi = MultiLabelBinarizer()
one_hot_multi.fit(posts['cat_multilabel'])
cat_multilabel = one_hot_multi.fit_transform(posts['cat_multilabel'])

#### Sentence vectors

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf = TfidfVectorizer(stop_words='english')

In [19]:
tfidf_matrix = tfidf.fit_transform(posts['title'])

In [20]:
tfidf_matrix, post_type_enc, np.array(cat_multilabel)

(<493x835 sparse matrix of type '<class 'numpy.float64'>'
 	with 1264 stored elements in Compressed Sparse Row format>,
 <1x493 sparse matrix of type '<class 'numpy.float64'>'
 	with 493 stored elements in Compressed Sparse Row format>,
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [21]:
from scipy.sparse import hstack
post_type_enc = post_type_enc.transpose()
hstack((tfidf_matrix, post_type_enc , cat_multilabel))

<493x1070 sparse matrix of type '<class 'numpy.float64'>'
	with 2659 stored elements in COOrdinate format>

In [22]:
# tfidf_matrix = np.concatenate((tfidf_matrix.toarray(), post_type_enc.toarray(), cat_multilabel), axis=0)

#### Similarity matrix

In [23]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [24]:
cosine_sim.shape

(493, 493)

In [25]:
s_matrix_df = pd.DataFrame(np.zeros(shape=(493,493)), columns=posts['_id'].unique(), index=posts['_id'].unique())
s_matrix_df

,5d62abaa65218653a132c956,5d6d39567fa40e1417a4931c,5d7d23315720533e15c3b1ee,5d7d405e5720533e15c3b1f3,5d80dfbc6c53455f896e600e,5d80e7c16c53455f896e6014,5d80ecfd6c53455f896e601a,5d81323a6c53455f896e6044,5d9b3514979d5962253c2f90,5d9b950768671220a1b2b153,...,5ed23cf876027d35905cc790,5ed23d4276027d35905cc798,5ed23e4d76027d35905cc7b8,5ed2502b76027d35905cc7db,5ed3476576027d35905cca1d,5ed3791976027d35905ccab6,5ed3820f76027d35905ccac8,5ed3ef4cbd514d602c1530f2,5ed415c6bd514d602c15312d,5ed4cbadbd514d602c1531a6
5d62abaa65218653a132c956,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d6d39567fa40e1417a4931c,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d7d23315720533e15c3b1ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d7d405e5720533e15c3b1f3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5d80dfbc6c53455f896e600e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5ed3791976027d35905ccab6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5ed3820f76027d35905ccac8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5ed3ef4cbd514d602c1530f2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5ed415c6bd514d602c15312d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
s_matrix_df[:] = cosine_sim
s_matrix_df

,5d62abaa65218653a132c956,5d6d39567fa40e1417a4931c,5d7d23315720533e15c3b1ee,5d7d405e5720533e15c3b1f3,5d80dfbc6c53455f896e600e,5d80e7c16c53455f896e6014,5d80ecfd6c53455f896e601a,5d81323a6c53455f896e6044,5d9b3514979d5962253c2f90,5d9b950768671220a1b2b153,...,5ed23cf876027d35905cc790,5ed23d4276027d35905cc798,5ed23e4d76027d35905cc7b8,5ed2502b76027d35905cc7db,5ed3476576027d35905cca1d,5ed3791976027d35905ccab6,5ed3820f76027d35905ccac8,5ed3ef4cbd514d602c1530f2,5ed415c6bd514d602c15312d,5ed4cbadbd514d602c1531a6
5d62abaa65218653a132c956,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
5d6d39567fa40e1417a4931c,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
5d7d23315720533e15c3b1ee,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
5d7d405e5720533e15c3b1f3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
5d80dfbc6c53455f896e600e,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5ed3791976027d35905ccab6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.166156,0.300071
5ed3820f76027d35905ccac8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.121305
5ed3ef4cbd514d602c1530f2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,1.0,0.000000,0.000000
5ed415c6bd514d602c15312d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.385402,0.0,0.166156,0.000000,0.0,1.000000,0.000000


In [27]:
blog_id = '5d6d39567fa40e1417a4931c'

In [30]:
# Use this function to generate recommendations

def get_recommendations(blog_id, s_matrix_df, posts, num):
    
    blog_indices = s_matrix_df.loc[blog_id].sort_values(ascending=False)[0:num][:].index.tolist()
    
    return posts[posts['_id'].isin(blog_indices)]['title']        

In [29]:
get_recommendations(blog_id, s_matrix_df, posts, 5)

1                                              Ml and AI
44             Will AI replace humans intellectual work?
170                                         Types Of AI.
224    10 Best Artificial Intelligence (AI) Courses O...
492    Configure CI/CD Pipeline in GitLab and deploym...
Name: title, dtype: object